In [1]:
from life_after_bert.data import load_olmpics_data, MCDataset
from life_after_bert.eval import evaluate_encoder

In [2]:
import numpy as np
import torch
import transformers

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# model_name = "bert-base-uncased"
model_name = "roberta-large"
task = "oLMpics MLM"
data_path = "../tests/data/oLMpics_age_comparison_train.jsonl"
num_choices = 2

In [4]:
model = transformers.AutoModelForMaskedLM.from_pretrained(model_name)
model.eval()
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNor

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

questions, choice_lists, answer_ids = load_olmpics_data(data_path, num_choices)
dataset = MCDataset(questions, choice_lists, answer_ids, tokenizer)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4032/4032 [00:00<00:00, 595871.67it/s]


In [6]:
all_answers, all_preds = evaluate_encoder(model, tokenizer, task, dataset, device)
(np.array(all_answers) == np.array(all_preds)).mean()

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 252/252 [00:07<00:00, 33.97it/s]


0.9397321428571429